In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
df = pd.read_csv("/Users/yonilevine/Desktop/coffee500.csv")

In [3]:
df.head()

,Unnamed: 0,url,title,roaster,roastlevel,blend,price,description,parse
0,0,https://one.mistobox.com/subscriptions/index/b...,"[<span class=""name"">Ethiopia Shara</span>]","[<div class=""roaster-description"">\n<!--<h2>Qu...","[<div class=""scale_circle light""></div>]","[<div class=""detail light active"">Light</div>,...",$20.00,"[<div class=""std"">Anodyne Coffee Roasting Co</...","[<div class=""mbox-description-content"">\n<!--<..."
1,1,https://one.mistobox.com/subscriptions/index/b...,"[<span class=""name"">Honduras David Mancia</span>]","[<div class=""roaster-description"">\n<!--<h2>Qu...","[<div class=""scale_circle medium_light""></div>]","[<div class=""detail medium_light sub_roast act...",$23.00,"[<div class=""std"">Coava Coffee Roasters</div>,...","[<div class=""mbox-description-content"">\n<!--<..."
2,2,https://one.mistobox.com/subscriptions/index/b...,"[<span class=""name"">Brazil Pedra Branca</span>]","[<div class=""roaster-description"">\n<!--<h2>Qu...","[<div class=""scale_circle light""></div>]","[<div class=""detail light active"">Light</div>,...",$23.00,"[<div class=""std"">Square One Coffee</div>, <di...","[<div class=""mbox-description-content"">\n<!--<..."
3,3,https://one.mistobox.com/subscriptions/index/b...,"[<span class=""name"">Kenya Ndaro-ini Peaberry</...","[<div class=""roaster-description"">\n<!--<h2>Qu...","[<div class=""scale_circle medium_light""></div>]","[<div class=""detail medium_light sub_roast act...",$23.00,"[<div class=""std"">AKA Coffee</div>, <div class...","[<div class=""mbox-description-content"">\n<!--<..."
4,4,https://one.mistobox.com/subscriptions/index/b...,"[<span class=""name"">Sumatra Permata Gayo</span>]","[<div class=""roaster-description"">\n<!--<h2>Qu...","[<div class=""scale_circle medium_dark""></div>]","[<div class=""detail medium_dark sub_roast acti...",$20.00,"[<div class=""std"">Higher Grounds Coffee</div>,...","[<div class=""mbox-description-content"">\n<!--<..."


In [4]:
df.drop('Unnamed: 0',axis =1 ,inplace=True)

In [5]:
def striphtml(data):
    p = re.compile(r'<.*?>')
    return p.sub('', data)

In [6]:
def striproast(data):
    p = re.compile(r'/([^,]+)/')
    return p.sub('', data)

In [7]:
def striphtml2(data):
    p = re.findall(r'\\n.*?\\n',data)
    return p

In [8]:
def striphtml3(data):
    p = re.findall(r'(?<=\\n)(.*)(?=</div>)',data)
    return p

In [9]:
def striphtml4(data):
    p = re.findall(r'(?<=Country\\n                                        </div>\\n)(.*)(?=</div>\\n</div>\\n</div>\\n</div)',data)
    return p

In [10]:
def striphtml5(data):
    p = re.findall(r'(?<=Altitude\\n                                        </div>\\n)(.*)(?=</div>\\n<div class="coffee-detail">\\n<div class="header variety">\\n<div class="icon"></div>\\n)',data)
    return p

In [11]:
def striphtml6(data):
    p = re.findall(r'(?<=Process\\n                                        </div>\\n)(.*)(?=</div>\\n<div class="coffee-detail">\\n<div class="header producer">\\n<div class="icon"></div>\\n)',data)
    return p

In [12]:
def striphtml7(data):
    p = re.sub("\\\\n", "", data)
    return p

In [13]:
def stripnum(data):
    return re.findall(r'\d+', data)

In [14]:
df['title'] = df['title'].apply(striphtml)
df['title'] = [row[1:-1] for row in df['title']]

In [15]:
df['roaster'] = df['roaster'].apply(striphtml)
df['roaster'] = [row[22:-3] for row in df['roaster']]

In [16]:
df['blend'] = df['blend'].apply(striphtml)
    

In [17]:
df['roastlevel'] = [row.split(',')[0][1:] for row in df['blend']]

In [18]:
df['blend'] = df.blend.apply(lambda s: s  + '  \\n                       Blend\\n' if len(s) < 25 else s) 

In [19]:
df['blend'] = df['blend'].apply(striphtml2)

In [20]:
df['description'] = df['description'].apply(striphtml3)

In [21]:
df['country'] = df['parse'].apply(striphtml4)

In [22]:
df['altitude'] = df['parse'].apply(striphtml5)

In [23]:
alt = list(df.altitude)
x=map(lambda x: list('q') if len(x)==0 else x, alt)
newlist = [item for items in x for item in items]
df['altitude']=newlist

In [24]:
df['altitude']=[item.replace('meters', '') for item in df['altitude']]
df['altitude']=[item.replace(',', '') for item in df['altitude']]
df['altitude']=[item.replace('+', '') for item in df['altitude']]
df['altitude']=[item.replace('-', ',') for item in df['altitude']]
df['altitude']=df.altitude.str.strip()

In [25]:
alt2=[]
for item in df.altitude:
    x=item.split(',')
    if len(x)==2:
        alt2.append(np.mean(float(x[0])+float(x[1]))/2)
    else:
        alt2.append(x[0])


In [26]:
df['altitude']=alt2

In [27]:
df['altitude'] = pd.to_numeric(df.altitude,errors='coerce')

In [28]:
df['process'] = df['process'] = df['parse'].apply(striphtml6)

In [29]:
df['process'] = df.process.astype(str).str.replace('\[|\]|\'', '')

In [30]:
df['description'] = df.description.astype(str).str.replace('\[|\]|\'', '')
df['country'] = df.country.astype(str).str.replace('\[|\]|\'', '')
df['blend'] = df.blend.astype(str).str.replace('\[|\]|\'', '')


In [31]:
df['blend']=df.blend.apply(striphtml7)

In [32]:
df.drop('parse',axis=1,inplace=True)

In [33]:
blend=[]
for item in df['blend']:
    blend.append(item[1:-1])

In [34]:
df['blend']=blend

In [35]:
blend2=[]
for sentence in df['blend']:
    blend2.append(sentence.replace(" ", ""))

In [36]:
df['blend']=blend2

In [37]:
def eda(dataframe):
    print("\nmissing values: \n", dataframe.isnull().sum())
    print("\ndataframe index: \n", dataframe.index)
    print("\ndataframe types: \n", dataframe.dtypes)
    print("\ndataframe shape: \n", dataframe.shape)
    print("\ndataframe describe: \n", dataframe.describe(), '\n')
    for item in dataframe:
        print(item + ':')
        print(dataframe[item].nunique())

In [38]:
country=[]
for item in df.country:
    country.append(item.partition('</div>\\')[0])
    

In [39]:
country2=[]
for item in country:
    country2.append(item.replace(" ", ""))

In [40]:
df['country']=country2

In [41]:
df['country']=df.country.str.strip()

In [42]:

df['country']=[item.replace(","," ") for item in df['country']]

In [43]:
df['roastlevel']=[item.strip() for item in df['roastlevel']]

In [44]:
df['roastlevel']=[item.replace("]"," ") for item in df['roastlevel']]

In [45]:
df['price']=df.price.str.strip("$")

In [90]:
df.to_csv("/Users/yonilevine/Desktop/datasets/coffeefinal.csv")

In [47]:
df.url.iloc[47]

'https://one.mistobox.com/subscriptions/index/buy?sku=KLCH-014'